In [8]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
# Load the envs var
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a comdedian who tells jokes about {topic}"),
        ("human", "Tell me {joke_count} jokes"),
    ]
)

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word Count: {len(x.split())}\n{x}")

chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

results = chain.invoke({"topic":"banker","joke_count":3})

print(results)

Word Count: 135
ALRIGHT, ALRIGHT, SETTLE DOWN FOLKS, YOU LOOK LIKE YOU'RE WAITING FOR A LOAN TO BE APPROVED. SPEAKING OF...

1.  WHY DID THE BANKER BREAK UP WITH THE MATHEMATICIAN? BECAUSE ALL HE EVER TALKED ABOUT WAS INTEREST! GET IT? INTEREST! LIKE MONEY...AND MATH! I'LL BE HERE ALL WEEK, TRY THE VEAL!

2.  A BANKER WALKS INTO A LIBRARY AND ASKS FOR BOOKS ABOUT PARANOIA. THE LIBRARIAN WHISPERS, "THEY'RE RIGHT BEHIND YOU!" THEY'RE ALWAYS LOOKING OVER THEIR SHOULDERS, THOSE GUYS! PROBABLY COUNTING THEIR MONEY...WHICH, LET'S BE HONEST, ISN'T THEIRS ANYWAY.

3.  WHAT'S THE DIFFERENCE BETWEEN A USED CAR SALESMAN AND A BANKER? THE USED CAR SALESMAN KNOWS WHEN HE'S LYING. BA-DUM-TISH! SERIOUSLY THOUGH, HAVE YOU *SEEN* THE FINE PRINT ON THOSE LOAN AGREEMENTS? YOU NEED A LAWYER AND A MAGNIFYING GLASS JUST TO UNDERSTAND WHAT YOU'RE SIGNING!


## Parallel Chains

In [14]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List me main features of {product_name}"),
    ]
)

# Define pros analysis step

def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system","You are an expert product reviwer"),
            ("human", "Given these features : {features}, list the pros of these features")
        ]
    )
    return pros_template.format_prompt(features=features)


# Define the cons step

def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system","You are an expert product reviwer"),
            ("human","Given these features: {features}, list the cons of these features")
        ]
    )
    return cons_template.format_prompt(features=features)


def combine_pros_cons(pros,cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"




In [18]:
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser() 
)
cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser() 
)

chain = (
    prompt_template |
    model |
    StrOutputParser() |
    RunnableParallel(branches={"pros":pros_branch_chain, "cons":cons_branch_chain}) |
    RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

results = chain.invoke({"product_name":"Macbook Pro"})

print(results)

Pros:
Okay, based on the feature list provided, here's a breakdown of the pros of each feature category for the MacBook Pro:

**Core Performance & Design:**

*   **Apple Silicon Processors (M-series):**
    *   **Pro:** Dramatically improved performance for demanding tasks like video editing, coding, and gaming.
    *   **Pro:** Significantly better power efficiency, leading to longer battery life.
    *   **Pro:** Runs cooler and quieter than previous Intel-based models.
*   **Unified Memory:**
    *   **Pro:** Faster data access and improved performance, especially for graphics-intensive applications.
    *   **Pro:** More efficient memory management, allowing for smoother multitasking.
*   **Liquid Retina XDR Display (on 14" and 16" models):**
    *   **Pro:** Exceptional image quality with vibrant colors, deep blacks, and incredible brightness.
    *   **Pro:** Ideal for content creators who need accurate color representation and high dynamic range.
    *   **Pro:** Promotes eye co